# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('zillow_data.csv')

In [ ]:
df.sort_values(by='RegionName').head(20)

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [ ]:
get_datetimes(df)

Note:  I have surmised that RegionName is the ZIP code for each entry.  RegionName values with only 4 digits represent ZIP codes that actually begin with '0'.  So that I can work with ZIP codes in the data set, I will need to add that zero onto every 4-digit RegionName value.  Once that's completed, I'll rename this column "ZipCode".  

In [ ]:
df.RegionName = df.RegionName.astype('int64')
df.RegionName.dtype

In [ ]:
# df.RegionName.sort_values().head(10)

df.sort_values(by="RegionName").head(20)

In [ ]:
df['RegionName'] = df.RegionName.astype(str)
df['RegionName'].dtypeb


In [ ]:

df_zips['Zip'] = df['RegionName']
df_zips.head()

In [ ]:
# df_zips['Zip'] = ['0' + i for i in df_zips['RegionName'] if len(i) < 5]

zips = []

for i in df_zips['RegionName']:
    if len(i) < 5:
        i = '0' + i
        zips.append(i)
    else:
        zips.append(i)

df_zips['Zip'] = pd.Series(zips)
df_zips['Zip'].sort_values().head(10)


In [ ]:
df_zips.sort_values(by='Zip').head()

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted   # removed remaining part of this code:  .groupby('time').aggregate({'value':'mean'})

In [ ]:
df_melt = melt_data(df)
df_melt.head()

In [ ]:
df_melt.groupby(['RegionName', 'time', 'State', 'City']).aggregate({'value':'mean'}).head(40)

In [ ]:
df_melt.groupby(['RegionName', 'time', 'State', 'City']).sum().head(40)

In [ ]:
df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'}).head()

In [ ]:
df_melt.info()

In [ ]:
df_melt.head()


In [ ]:
df_melt.set_index('time', inplace=True)

In [ ]:
df_melt.head(10)

In [ ]:
df_melt.loc[df_melt['State'] == 'AK'].groupby(['time']).aggregate({'value':'mean'}).head()

In [ ]:
df_melt['1997':].head(10)
df_melt.loc[df_melt['State'] == 'MA'].groupby(['RegionID', 'RegionName', 'time']).mean()

In [ ]:
# df_melt_annual = df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'})
# df_melt_annual.loc[df_melt_annual['City'] =='Chicago'].aggregate({'value':'mean'})
                                # .groupby(['time']).aggregate({'value':'mean'}).head()
df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'})
df_melt.loc[df_melt['City']=='Chicago']

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}

matplotlib.rc('font', **font)

df.plot(figsize=(12, 6));

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results